# 안녕하세요^^ 
# AIVLE 미니 프로젝트에 오신 여러분을 환영합니다.
* 본 과정에서는 실제 사례와 데이터를 기반으로 문제를 해결하는 전체 과정을 자기 주도형 실습으로 진행해볼 예정입니다.
* 앞선 교육과정을 정리하는 마음과 지금까지 배운 내용을 바탕으로 문제 해결을 해볼게요!
* 미니 프로젝트를 통한 문제 해결 과정 'A에서 Z까지', 지금부터 시작합니다!

---

In [54]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import * 

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings
warnings.filterwarnings(action='ignore')

from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RepeatedKFold, cross_val_score, StratifiedKFold, KFold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from datetime import datetime, timedelta

---

# (실습준비) 데이터 불러오기부터

In [55]:
# 학습 데이터 로딩

air_21 = pd.read_csv("air_2021.csv", sep=',', index_col = 0, encoding = 'utf-8' )
air_22 = pd.read_csv("air_2022.csv", sep=',', index_col = 0, encoding = 'utf-8' )


# from google.colab import drive
# drive.mount('/content/drive')
# air_21 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mini-project-2/air_2021.csv", sep=',', index_col = 0, encoding = 'utf-8' )
# air_22 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mini-project-2/air_2022.csv", sep=',', index_col = 0, encoding = 'utf-8' )

In [56]:
weather_21 = pd.read_csv("weather_2021.csv", sep = ',', encoding='cp949')
weather_22 = pd.read_csv("weather_2022.csv", sep = ',', encoding='cp949')

# weather_21 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mini-project-2/weather_2021.csv", sep = ',', encoding='cp949')
# weather_22 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/mini-project-2/weather_2022.csv", sep = ',', encoding='cp949')

---

# 1. 이상치/결측치 처리

#### [실습문제1] 미세먼지 데이터 결측치 처리
* 미세먼지 데이터의 결측치를 확인하고 결측치가 존재한다면 처리해주세요.


* 앞서 데이터 불러오기 및 분석 단계에서 확인한 불필요한 변수에 대해 제거 해 주세요. 

In [57]:
# group할 일시 제외 불필요 변수 drop
air_21 = air_21.drop(columns = ['지역', '망', '측정소코드', '측정소명', '주소'])
air_22 = air_22.drop(columns = ['지역', '망', '측정소코드', '측정소명', '주소'])

In [58]:
air_21.head()

,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,2021100101,0.003,0.6,0.002,0.039,31.0,18.0
1,2021100102,0.003,0.6,0.002,0.035,27.0,16.0
2,2021100103,0.003,0.6,0.002,0.033,28.0,18.0
3,2021100104,0.003,0.6,0.002,0.030,26.0,16.0
4,2021100105,0.003,0.5,0.003,0.026,26.0,16.0


In [59]:
air_22.head()

,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,2022010101,0.003,0.4,0.026,0.016,23.0,12.0
1,2022010102,0.003,0.4,0.022,0.020,20.0,9.0
2,2022010103,0.003,0.5,0.014,0.028,20.0,9.0
3,2022010104,0.003,0.5,0.016,0.027,19.0,10.0
4,2022010105,0.003,0.5,0.005,0.040,24.0,11.0


In [60]:
imp = IterativeImputer(estimator=RandomForestRegressor())

In [61]:
imp.fit(air_21)

IterativeImputer(estimator=RandomForestRegressor())

In [62]:
air_21 = pd.DataFrame(imp.transform(air_21), columns = air_21.columns)

In [63]:
air_21.isna().sum()

측정일시    0
SO2     0
CO      0
O3      0
NO2     0
PM10    0
PM25    0
dtype: int64

In [64]:
imp.fit(air_22)

IterativeImputer(estimator=RandomForestRegressor())

In [65]:
air_22 = pd.DataFrame(imp.transform(air_22), columns = air_22.columns)

In [66]:
air_22.isna().sum()

측정일시    0
SO2     0
CO      0
O3      0
NO2     0
PM10    0
PM25    0
dtype: int64

#### [실습문제2] 날씨 데이터 결측치 처리
* 날씨 데이터의 결측치를 확인하고 결측치가 존재한다면 처리해주세요.

* '강우량'을 어떻게 해야할까요?

- 강우량 : 0에 대한 값이 따로 있는 것으로 보아 장비 점검 오류 등으로 인한 미측정 구간으로 예상, 역시 IterativeImputer로 결측치 처리 필요

In [67]:
columns = ['지점', 
           '지점명', 
           '기온 QC플래그',
           '강수량 QC플래그',
           '풍속 QC플래그',
           '풍향 QC플래그',
           '습도 QC플래그',
           '현지기압 QC플래그',
           '해면기압 QC플래그',
           '일조 QC플래그',
           '일사 QC플래그',
           '운형(운형약어)',
           '지면상태(지면상태코드)',
           '현상번호(국내식)',
           '지면온도 QC플래그']

weather_21 = weather_21.drop(columns = columns)
weather_22 = weather_22.drop(columns = columns)

In [68]:
weather_21.head()

,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),...,3시간신적설(cm),전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,2021-01-01 01:00,-8.7,NaN,2.4,270.0,68,2.2,-13.5,1016.4,1027.7,...,NaN,0.0,0,NaN,2000,-6.9,-1.0,-0.8,0.3,1.6
1,2021-01-01 02:00,-9.1,NaN,1.6,270.0,69,2.1,-13.7,1016.2,1027.5,...,NaN,0.0,0,NaN,2000,-7.1,-1.1,-0.8,0.3,1.6
2,2021-01-01 03:00,-9.3,NaN,1.1,250.0,70,2.1,-13.7,1016.8,1028.1,...,NaN,0.0,0,NaN,2000,-7.3,-1.2,-0.9,0.3,1.6
3,2021-01-01 04:00,-9.3,NaN,0.3,0.0,71,2.2,-13.5,1016.2,1027.5,...,NaN,0.0,0,NaN,2000,-7.5,-1.3,-1.0,0.2,1.5
4,2021-01-01 05:00,-9.7,NaN,1.9,20.0,72,2.1,-13.8,1015.6,1026.9,...,NaN,0.0,0,NaN,2000,-7.6,-1.3,-1.0,0.2,1.5


In [69]:
#imp = IterativeImputer(estimator=RandomForestRegressor())

In [70]:
weather_21.interpolate(method = 'linear', inplace = True) 

In [71]:
weather_21 = weather_21.fillna(0)

In [72]:
weather_21

,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),...,3시간신적설(cm),전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,2021-01-01 01:00,-8.7,0.0,2.4,270.0,68,2.2,-13.5,1016.4,1027.7,...,0.0,0.0,0,0.0,2000,-6.9,-1.0,-0.8,0.3,1.6
1,2021-01-01 02:00,-9.1,0.0,1.6,270.0,69,2.1,-13.7,1016.2,1027.5,...,0.0,0.0,0,0.0,2000,-7.1,-1.1,-0.8,0.3,1.6
2,2021-01-01 03:00,-9.3,0.0,1.1,250.0,70,2.1,-13.7,1016.8,1028.1,...,0.0,0.0,0,0.0,2000,-7.3,-1.2,-0.9,0.3,1.6
3,2021-01-01 04:00,-9.3,0.0,0.3,0.0,71,2.2,-13.5,1016.2,1027.5,...,0.0,0.0,0,0.0,2000,-7.5,-1.3,-1.0,0.2,1.5
4,2021-01-01 05:00,-9.7,0.0,1.9,20.0,72,2.1,-13.8,1015.6,1026.9,...,0.0,0.0,0,0.0,2000,-7.6,-1.3,-1.0,0.2,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,2021-12-31 19:00,-6.4,0.0,2.2,250.0,34,1.3,-19.7,1021.1,1032.3,...,0.3,0.0,0,10.0,2000,-4.5,-0.5,-0.7,-0.1,1.0
8755,2021-12-31 20:00,-6.3,0.0,4.1,320.0,35,1.3,-19.2,1021.2,1032.4,...,0.3,0.0,0,10.0,2000,-5.3,-0.6,-0.8,-0.1,1.0
8756,2021-12-31 21:00,-6.7,0.0,4.8,320.0,36,1.3,-19.3,1021.2,1032.4,...,0.3,0.0,0,10.0,2000,-5.7,-0.7,-0.8,-0.1,1.0
8757,2021-12-31 22:00,-7.5,0.0,3.0,320.0,37,1.3,-19.7,1021.8,1033.1,...,0.3,0.0,0,10.0,2000,-6.2,-0.8,-0.9,-0.2,1.0


In [73]:
weather_22.interpolate(method = 'linear', inplace = True) 

In [74]:
weather_22 = weather_22.fillna(0)

In [75]:
#imp.fit(weather_22)

In [76]:
#weather_22 = pd.DataFrame(imp.transform(weather_22), columns = weather_22.columns)

In [77]:
weather_22.isna().sum()

일시               0
기온(°C)           0
강수량(mm)          0
풍속(m/s)          0
풍향(16방위)         0
습도(%)            0
증기압(hPa)         0
이슬점온도(°C)        0
현지기압(hPa)        0
해면기압(hPa)        0
일조(hr)           0
일사(MJ/m2)        0
적설(cm)           0
3시간신적설(cm)       0
전운량(10분위)        0
중하층운량(10분위)      0
최저운고(100m )      0
시정(10m)          0
지면온도(°C)         0
5cm 지중온도(°C)     0
10cm 지중온도(°C)    0
20cm 지중온도(°C)    0
30cm 지중온도(°C)    0
dtype: int64

#### [실습문제3] air_21, air_22 의 '측정일시'를 활용하여 'time'변수 생성하기


* air_21, air_22 의 '측정일시'를 활용하여 'time'변수 생성


* time 변수를 datatime으로 데이터 타입 변경하기

In [78]:
air_21['측정일시'] = air_21['측정일시'].astype(str)

In [79]:
air_21

,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,2021100101.0,0.003,0.6,0.002,0.039,31.0,18.0
1,2021100102.0,0.003,0.6,0.002,0.035,27.0,16.0
2,2021100103.0,0.003,0.6,0.002,0.033,28.0,18.0
3,2021100104.0,0.003,0.6,0.002,0.030,26.0,16.0
4,2021100105.0,0.003,0.5,0.003,0.026,26.0,16.0
...,...,...,...,...,...,...,...
8755,2021093020.0,0.003,0.7,0.020,0.036,35.0,24.0
8756,2021093021.0,0.003,0.6,0.016,0.035,34.0,21.0
8757,2021093022.0,0.003,0.6,0.012,0.036,30.0,19.0
8758,2021093023.0,0.003,0.6,0.004,0.042,33.0,19.0


In [80]:
converted_time = air_21['측정일시'].str.slice(stop =-4) + air_21['측정일시'].str.slice(start=-4).map(lambda x: str(int(x[0:2])-1).zfill(2))

In [81]:
air_21['측정일시'] = pd.to_datetime(converted_time, format ='%Y%m%d%H')

In [82]:
air_21

,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,2021-10-01 00:00:00,0.003,0.6,0.002,0.039,31.0,18.0
1,2021-10-01 01:00:00,0.003,0.6,0.002,0.035,27.0,16.0
2,2021-10-01 02:00:00,0.003,0.6,0.002,0.033,28.0,18.0
3,2021-10-01 03:00:00,0.003,0.6,0.002,0.030,26.0,16.0
4,2021-10-01 04:00:00,0.003,0.5,0.003,0.026,26.0,16.0
...,...,...,...,...,...,...,...
8755,2021-09-30 19:00:00,0.003,0.7,0.020,0.036,35.0,24.0
8756,2021-09-30 20:00:00,0.003,0.6,0.016,0.035,34.0,21.0
8757,2021-09-30 21:00:00,0.003,0.6,0.012,0.036,30.0,19.0
8758,2021-09-30 22:00:00,0.003,0.6,0.004,0.042,33.0,19.0


In [83]:
# 아래에 실습코드를 작성하고 결과를 확인합니다.
air_22['측정일시'] = air_22['측정일시'].astype(str)
converted_time = air_22['측정일시'].str.slice(stop =-4) + air_22['측정일시'].str.slice(start=-4).map(lambda x: str(int(x[0:2])-1).zfill(2))

In [84]:
air_22['측정일시'] = pd.to_datetime(converted_time, format ='%Y%m%d%H')

In [85]:
air_22

,측정일시,SO2,CO,O3,NO2,PM10,PM25
0,2022-01-01 00:00:00,0.003,0.4,0.026,0.016,23.0,12.0
1,2022-01-01 01:00:00,0.003,0.4,0.022,0.020,20.0,9.0
2,2022-01-01 02:00:00,0.003,0.5,0.014,0.028,20.0,9.0
3,2022-01-01 03:00:00,0.003,0.5,0.016,0.027,19.0,10.0
4,2022-01-01 04:00:00,0.003,0.5,0.005,0.040,24.0,11.0
...,...,...,...,...,...,...,...
2155,2022-03-31 19:00:00,0.002,0.4,0.036,0.017,11.0,8.0
2156,2022-03-31 20:00:00,0.002,0.4,0.032,0.018,10.0,7.0
2157,2022-03-31 21:00:00,0.003,0.3,0.038,0.013,11.0,5.0
2158,2022-03-31 22:00:00,0.003,0.3,0.036,0.015,11.0,5.0


In [86]:
air_21 = air_21.rename(columns = {'측정일시': 'time'})
air_22 = air_22.rename(columns = {'측정일시': 'time'})

#### [실습문제4] weather_21, weather_22 의 '일시'를 활용하여 'time'변수 생성하기


* weather_21, weather_22 의 '측정일시'를 활용하여 'time'변수 생성


* time 변수를 datatime으로 데이터 타입 변경하기

In [87]:
weather_21['time']=weather_21['일시']
weather_21['time']=pd.to_datetime(weather_21['time'])

In [88]:
weather_22['time']=weather_22['일시']
weather_22['time']=pd.to_datetime(weather_22['time'])

In [89]:
weather_21

,일시,기온(°C),강수량(mm),풍속(m/s),풍향(16방위),습도(%),증기압(hPa),이슬점온도(°C),현지기압(hPa),해면기압(hPa),...,전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C),time
0,2021-01-01 01:00,-8.7,0.0,2.4,270.0,68,2.2,-13.5,1016.4,1027.7,...,0.0,0,0.0,2000,-6.9,-1.0,-0.8,0.3,1.6,2021-01-01 01:00:00
1,2021-01-01 02:00,-9.1,0.0,1.6,270.0,69,2.1,-13.7,1016.2,1027.5,...,0.0,0,0.0,2000,-7.1,-1.1,-0.8,0.3,1.6,2021-01-01 02:00:00
2,2021-01-01 03:00,-9.3,0.0,1.1,250.0,70,2.1,-13.7,1016.8,1028.1,...,0.0,0,0.0,2000,-7.3,-1.2,-0.9,0.3,1.6,2021-01-01 03:00:00
3,2021-01-01 04:00,-9.3,0.0,0.3,0.0,71,2.2,-13.5,1016.2,1027.5,...,0.0,0,0.0,2000,-7.5,-1.3,-1.0,0.2,1.5,2021-01-01 04:00:00
4,2021-01-01 05:00,-9.7,0.0,1.9,20.0,72,2.1,-13.8,1015.6,1026.9,...,0.0,0,0.0,2000,-7.6,-1.3,-1.0,0.2,1.5,2021-01-01 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8754,2021-12-31 19:00,-6.4,0.0,2.2,250.0,34,1.3,-19.7,1021.1,1032.3,...,0.0,0,10.0,2000,-4.5,-0.5,-0.7,-0.1,1.0,2021-12-31 19:00:00
8755,2021-12-31 20:00,-6.3,0.0,4.1,320.0,35,1.3,-19.2,1021.2,1032.4,...,0.0,0,10.0,2000,-5.3,-0.6,-0.8,-0.1,1.0,2021-12-31 20:00:00
8756,2021-12-31 21:00,-6.7,0.0,4.8,320.0,36,1.3,-19.3,1021.2,1032.4,...,0.0,0,10.0,2000,-5.7,-0.7,-0.8,-0.1,1.0,2021-12-31 21:00:00
8757,2021-12-31 22:00,-7.5,0.0,3.0,320.0,37,1.3,-19.7,1021.8,1033.1,...,0.0,0,10.0,2000,-6.2,-0.8,-0.9,-0.2,1.0,2021-12-31 22:00:00


In [90]:
col1=weather_21.columns[-1:].to_list()
col2=weather_21.columns[:-1].to_list()
new_col=col1+col2
weather_21=weather_21[new_col]

In [91]:
col1=weather_22.columns[-1:].to_list()
col2=weather_22.columns[:-1].to_list()
new_col=col1+col2
weather_22=weather_22[new_col]

In [92]:
weather_21.drop(columns = ['일시'], inplace = True)
weather_22.drop(columns = ['일시'], inplace = True)

---

# 2. 데이터 합치기
* 미세먼지 데이터와 날씨 데이터를 'time' 기준으로 합쳐보겠습니다

#### [실습문제5] 데이터 합치기
* 'df_21','df_22'에 21년도와 22년도의 미세먼지, 날씨 데이터를 합쳐 넣도록 하겠습니다.

#### air 21, 22 시간순으로 정렬

In [93]:
air_21.sort_values(by=['time'], axis=0, inplace=True)
air_22.sort_values(by=['time'], axis=0, inplace=True)

#### train set 병합

In [94]:
df_21 =  pd.merge(air_21, weather_21, how='outer',on='time')

In [95]:
df_21.head()

,time,SO2,CO,O3,NO2,PM10,PM25,기온(°C),강수량(mm),풍속(m/s),...,3시간신적설(cm),전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,2021-01-01 00:00:00,0.002,0.5,0.022,0.016,24.0,14.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01 01:00:00,0.002,0.6,0.018,0.020,25.0,14.0,-8.7,0.0,2.4,...,0.0,0.0,0.0,0.0,2000.0,-6.9,-1.0,-0.8,0.3,1.6
2,2021-01-01 02:00:00,0.002,0.6,0.013,0.025,27.0,16.0,-9.1,0.0,1.6,...,0.0,0.0,0.0,0.0,2000.0,-7.1,-1.1,-0.8,0.3,1.6
3,2021-01-01 03:00:00,0.003,0.6,0.011,0.027,23.0,13.0,-9.3,0.0,1.1,...,0.0,0.0,0.0,0.0,2000.0,-7.3,-1.2,-0.9,0.3,1.6
4,2021-01-01 04:00:00,0.003,0.6,0.008,0.032,24.0,14.0,-9.3,0.0,0.3,...,0.0,0.0,0.0,0.0,2000.0,-7.5,-1.3,-1.0,0.2,1.5


#### test set 병합

In [96]:
df_22 = pd.merge(air_22, weather_22, how='outer',on='time')

In [97]:
df_22.head()

,time,SO2,CO,O3,NO2,PM10,PM25,기온(°C),강수량(mm),풍속(m/s),...,3시간신적설(cm),전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,2022-01-01 00:00:00,0.003,0.4,0.026,0.016,23.0,12.0,-8.5,0.0,1.9,...,0.0,0,0,0.0,2000,-7.0,-1.0,-1.0,-0.2,1.0
1,2022-01-01 01:00:00,0.003,0.4,0.022,0.020,20.0,9.0,-9.2,0.0,1.8,...,0.0,0,0,0.0,2000,-7.2,-1.1,-1.1,-0.2,1.0
2,2022-01-01 02:00:00,0.003,0.5,0.014,0.028,20.0,9.0,-9.5,0.0,1.2,...,0.0,0,0,0.0,2000,-7.5,-1.3,-1.2,-0.2,1.0
3,2022-01-01 03:00:00,0.003,0.5,0.016,0.027,19.0,10.0,-9.3,0.0,1.4,...,0.0,0,0,0.0,2000,-7.6,-1.4,-1.2,-0.2,1.0
4,2022-01-01 04:00:00,0.003,0.5,0.005,0.040,24.0,11.0,-9.6,0.0,1.7,...,0.0,0,0,0.0,2000,-7.6,-1.4,-1.3,-0.3,1.0


### nan값 처리

In [98]:
df_21.interpolate(method = 'bfill', inplace = True )
df_21.isna().sum()

time             0
SO2              0
CO               0
O3               0
NO2              0
PM10             0
PM25             0
기온(°C)           0
강수량(mm)          0
풍속(m/s)          0
풍향(16방위)         0
습도(%)            0
증기압(hPa)         0
이슬점온도(°C)        0
현지기압(hPa)        0
해면기압(hPa)        0
일조(hr)           0
일사(MJ/m2)        0
적설(cm)           0
3시간신적설(cm)       0
전운량(10분위)        0
중하층운량(10분위)      0
최저운고(100m )      0
시정(10m)          0
지면온도(°C)         0
5cm 지중온도(°C)     0
10cm 지중온도(°C)    0
20cm 지중온도(°C)    0
30cm 지중온도(°C)    0
dtype: int64

In [99]:
df_22.interpolate(method = 'bfill', inplace = True)
df_22.isna().sum()

time             0
SO2              0
CO               0
O3               0
NO2              0
PM10             0
PM25             0
기온(°C)           0
강수량(mm)          0
풍속(m/s)          0
풍향(16방위)         0
습도(%)            0
증기압(hPa)         0
이슬점온도(°C)        0
현지기압(hPa)        0
해면기압(hPa)        0
일조(hr)           0
일사(MJ/m2)        0
적설(cm)           0
3시간신적설(cm)       0
전운량(10분위)        0
중하층운량(10분위)      0
최저운고(100m )      0
시정(10m)          0
지면온도(°C)         0
5cm 지중온도(°C)     0
10cm 지중온도(°C)    0
20cm 지중온도(°C)    0
30cm 지중온도(°C)    0
dtype: int64

In [100]:
df_21.head()

,time,SO2,CO,O3,NO2,PM10,PM25,기온(°C),강수량(mm),풍속(m/s),...,3시간신적설(cm),전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,2021-01-01 00:00:00,0.002,0.5,0.022,0.016,24.0,14.0,-8.7,0.0,2.4,...,0.0,0.0,0.0,0.0,2000.0,-6.9,-1.0,-0.8,0.3,1.6
1,2021-01-01 01:00:00,0.002,0.6,0.018,0.020,25.0,14.0,-8.7,0.0,2.4,...,0.0,0.0,0.0,0.0,2000.0,-6.9,-1.0,-0.8,0.3,1.6
2,2021-01-01 02:00:00,0.002,0.6,0.013,0.025,27.0,16.0,-9.1,0.0,1.6,...,0.0,0.0,0.0,0.0,2000.0,-7.1,-1.1,-0.8,0.3,1.6
3,2021-01-01 03:00:00,0.003,0.6,0.011,0.027,23.0,13.0,-9.3,0.0,1.1,...,0.0,0.0,0.0,0.0,2000.0,-7.3,-1.2,-0.9,0.3,1.6
4,2021-01-01 04:00:00,0.003,0.6,0.008,0.032,24.0,14.0,-9.3,0.0,0.3,...,0.0,0.0,0.0,0.0,2000.0,-7.5,-1.3,-1.0,0.2,1.5


In [101]:
df_22.head()

,time,SO2,CO,O3,NO2,PM10,PM25,기온(°C),강수량(mm),풍속(m/s),...,3시간신적설(cm),전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,2022-01-01 00:00:00,0.003,0.4,0.026,0.016,23.0,12.0,-8.5,0.0,1.9,...,0.0,0,0,0.0,2000,-7.0,-1.0,-1.0,-0.2,1.0
1,2022-01-01 01:00:00,0.003,0.4,0.022,0.020,20.0,9.0,-9.2,0.0,1.8,...,0.0,0,0,0.0,2000,-7.2,-1.1,-1.1,-0.2,1.0
2,2022-01-01 02:00:00,0.003,0.5,0.014,0.028,20.0,9.0,-9.5,0.0,1.2,...,0.0,0,0,0.0,2000,-7.5,-1.3,-1.2,-0.2,1.0
3,2022-01-01 03:00:00,0.003,0.5,0.016,0.027,19.0,10.0,-9.3,0.0,1.4,...,0.0,0,0,0.0,2000,-7.6,-1.4,-1.2,-0.2,1.0
4,2022-01-01 04:00:00,0.003,0.5,0.005,0.040,24.0,11.0,-9.6,0.0,1.7,...,0.0,0,0,0.0,2000,-7.6,-1.4,-1.3,-0.3,1.0


#### [실습문제7] 미세먼지 농도 다음시점 데이터 생성

* t+1 시점 데이터 생성

In [102]:
df_21['PM10+1'] = df_21['PM10'].shift(-1)
df_21.dropna(axis = 0, inplace = True)
df_21.tail()

,time,SO2,CO,O3,NO2,PM10,PM25,기온(°C),강수량(mm),풍속(m/s),...,전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C),PM10+1
8754,2021-12-31 18:00:00,0.003,0.4,0.026,0.019,25.0,10.0,-5.9,0.0,2.2,...,0.0,0.0,10.0,2000.0,-3.4,-0.3,-0.6,-0.1,1.0,26.0
8755,2021-12-31 19:00:00,0.003,0.4,0.025,0.020,26.0,8.0,-6.4,0.0,2.2,...,0.0,0.0,10.0,2000.0,-4.5,-0.5,-0.7,-0.1,1.0,27.0
8756,2021-12-31 20:00:00,0.003,0.4,0.030,0.014,27.0,9.0,-6.3,0.0,4.1,...,0.0,0.0,10.0,2000.0,-5.3,-0.6,-0.8,-0.1,1.0,20.0
8757,2021-12-31 21:00:00,0.003,0.4,0.033,0.011,20.0,8.0,-6.7,0.0,4.8,...,0.0,0.0,10.0,2000.0,-5.7,-0.7,-0.8,-0.1,1.0,20.0
8758,2021-12-31 22:00:00,0.003,0.4,0.029,0.015,20.0,9.0,-7.5,0.0,3.0,...,0.0,0.0,10.0,2000.0,-6.2,-0.8,-0.9,-0.2,1.0,20.0


In [103]:
df_22['PM10+1'] = df_22['PM10'].shift(-1)
df_22.dropna(axis = 0, inplace = True)
df_22.tail()

,time,SO2,CO,O3,NO2,PM10,PM25,기온(°C),강수량(mm),풍속(m/s),...,전운량(10분위),중하층운량(10분위),최저운고(100m ),시정(10m),지면온도(°C),5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C),PM10+1
2154,2022-03-31 18:00:00,0.002,0.4,0.044,0.010,10.0,9.0,13.2,0.0,4.3,...,9,0,12.5,2000,12.4,12.9,12.0,10.8,10.1,11.0
2155,2022-03-31 19:00:00,0.002,0.4,0.036,0.017,11.0,8.0,12.3,0.0,2.9,...,6,1,9.0,2000,10.0,12.6,11.8,11.0,10.2,10.0
2156,2022-03-31 20:00:00,0.002,0.4,0.032,0.018,10.0,7.0,11.6,0.0,2.7,...,6,0,9.5,2000,8.9,12.2,11.6,11.0,10.3,11.0
2157,2022-03-31 21:00:00,0.003,0.3,0.038,0.013,11.0,5.0,10.5,0.0,3.5,...,1,1,10.0,2000,7.8,11.8,11.4,11.0,10.4,11.0
2158,2022-03-31 22:00:00,0.003,0.3,0.036,0.015,11.0,5.0,9.3,0.0,1.9,...,3,3,10.0,2000,7.3,11.4,11.1,10.9,10.5,9.0


In [104]:
df_21.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8759 entries, 0 to 8758
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   time           8759 non-null   datetime64[ns]
 1   SO2            8759 non-null   float64       
 2   CO             8759 non-null   float64       
 3   O3             8759 non-null   float64       
 4   NO2            8759 non-null   float64       
 5   PM10           8759 non-null   float64       
 6   PM25           8759 non-null   float64       
 7   기온(°C)         8759 non-null   float64       
 8   강수량(mm)        8759 non-null   float64       
 9   풍속(m/s)        8759 non-null   float64       
 10  풍향(16방위)       8759 non-null   float64       
 11  습도(%)          8759 non-null   float64       
 12  증기압(hPa)       8759 non-null   float64       
 13  이슬점온도(°C)      8759 non-null   float64       
 14  현지기압(hPa)      8759 non-null   float64       
 15  해면기압(hPa)      8759 n

# 3. train / test split

#### [실습문제7] 데이터셋 나누기 

* 데이터셋을 train(21년도 데이터), test(22년도 데이터)로 나눠 csv로 저장하도록 하겠습니다.


* train_x / train_y / test_x / test_y 로 나누세요.


* train_x.csv / train_y.csv / test_x.csv / test_y.csv 로 저장하세요.

In [105]:
from sklearn.model_selection import train_test_split

target = 'PM10+1'
x = df_21.drop([target, 'time'], axis = 1) #제거할 때, date도 제거
y = df_21.loc[:, target]

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.05, shuffle = False)


x.to_csv('df_21_feature.csv', encoding='utf-8-sig', index=False)
y.to_csv('df_21_target.csv', encoding='utf-8-sig', index=False)

x_train.to_csv('df_21_feature_x_train.csv', encoding='utf-8-sig', index=False)
x_val.to_csv('df_21_feature_x_val.csv', encoding='utf-8-sig', index=False)
y_train.to_csv('df_21_target_y_train.csv', encoding='utf-8-sig', index=False)
y_val.to_csv('df_21_target_y_val.csv', encoding='utf-8-sig', index=False)




# x.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_21_feature.csv', encoding='utf-8-sig', index=False)
# y.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_21_target.csv', encoding='utf-8-sig', index=False)

# x_train.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_21_feature_x_train.csv', encoding='utf-8-sig', index=False)
# x_val.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_21_feature_x_val.csv', encoding='utf-8-sig', index=False)
# y_train.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_21_target_y_train.csv', encoding='utf-8-sig', index=False)
# y_val.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_21_target_y_val.csv', encoding='utf-8-sig', index=False)

In [106]:
target = 'PM10+1'
x = df_22.drop([target, 'time'], axis = 1) #제거할 때, date도 제거
y = df_22.loc[:, target]



x.to_csv('df_22_feature.csv', encoding='utf-8-sig', index=False)
y.to_csv('df_22_target.csv', encoding='utf-8-sig', index=False)


# x.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_22_feature.csv', encoding='utf-8-sig', index=False)
# y.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_22_target.csv', encoding='utf-8-sig', index=False)

# x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = 0.05, shuffle = False)

# x_train.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_22_feature.csv', encoding='utf-8-sig')
# x_val.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_22_test_feature.csv', encoding='utf-8-sig')
# y_train.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_22_target.csv', encoding='utf-8-sig')
# y_val.to_csv('/content/drive/MyDrive/Colab Notebooks/mini-project-2/df_22_test_target.csv', encoding='utf-8-sig')

---